In [1]:
import os
import pandas as pd
import numpy as np
import values, objects, helpers
import uuid
import random
from datetime import datetime, timedelta

##### Tables

In [2]:
cdhdr_cols = ['MANDT', 'OBJECTCLAS', 'OBJECTID', 'CHANGENR', 'USERNAME', 'ERDAT']
cdpos_cols = ['MANDT', 'CHANGENR', 'TABNAME', 'FNAME', 'OLDVALUE', 'NEWVALUE']
mara_cols = ['MANDT', 'MATNR', 'MAKTX']
usr01_cols = ['MANDT', 'BNAME', 'UTYPE']
Kna1_cols = ['MANDT', 'KUNNR', 'NAME1', 'ERDAT', 'ERNAM']
vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'VSBED', 'FAKSK']
# vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'shp. cond.', 'bil. blk.']
vbap_cols = ['MANDT', 'VBELN', 'POSNR', 'KWMENG', 'MATNR', 'NETWR']
likp_cols = ['MANDT', 'VBELN', 'WADAT', 'LFUHR', 'ERDAT', 'ERNAM']
# likp_cols = ['MANDT', 'id', 'planned. del. date', 'del. date', 'ERDAT', 'ERNAM']
lips_cols = ['MANDT', 'VBELN', 'KDAUF', 'POSNR']
# lips_cols = ['MANDT', 'likp.id', 'vbap.vbeln', 'vbap.posnr']
vbuk_cols = ['MANDT', 'VBELN', 'LFSTK', 'FKSTK', 'AEDAT', 'VBTYP']
# vbuk_cols = ['MANDT', 'VBELN', 'del. stat', 'bil. stat', 'changed on', 'sd cat.']
mkpf_cols = ['MANDT', 'MBLNR', 'BLART', 'AEDAT', 'USNAM']
# mkpf_cols = ['MANDT', 'id', 'BLART', 'AEDAT', 'USNAM']
mseg_cols = ['MANDT', 'MBLNR', 'KDAUF', 'KDPOS']
# mseg_cols = ['MANDT', 'mkpf.id', 'vbap.vbeln', 'vbap.posnr']
vbrk_cols = ['MANDT', 'VBELN', 'VBTYP', 'ERNAM', 'ERDAT']
# vbrk_cols = ['MANDT', 'id', 'VBTYP', 'ERNAM', 'ERDAT']
vbrp_cols = ['MANDT', 'VBELN', 'AUBEL', 'POSNR']
# vbrp_cols = ['MANDT', 'id', 'vbak.vbeln', 'vbap.posnr']

In [3]:
vbak_len = 100

In [4]:
# Materials
mara_temp_list = []
MATERIALS = []
for mat in values.material_names:
    rand_id = uuid.uuid4()
    mara_temp_list.append([values.mandt, rand_id, mat['name']])
    MATERIALS.append(objects.Materials(id=rand_id, name=mat['name'], availability=mat['availability'], price=mat['price']))

mara =  pd.DataFrame(mara_temp_list, columns=mara_cols)

In [5]:
# Users
usr01_temp_list = []
USERS = []
for u_name in values.users:
    usr01_temp_list.append([values.mandt, u_name, 'A'])
    USERS.append(objects.Users(u_name, 'A'))
usr01_temp_list.append([values.mandt, 'BATCH_JOB', 'B'])
USERS.append(objects.Users('BATCH_JOB', 'B'))

usr01 = pd.DataFrame(usr01_temp_list, columns=usr01_cols)

In [6]:
# Customers
kna1_temp_list = []
COMPANIES = []
for company in values.company_names:
    rand_id = uuid.uuid4()
    rand_dt = values.times['customer_create_start'] + helpers.UPTO_YEAR()
    rand_usr = random.choice(list(values.users))
    kna1_temp_list.append([values.mandt, rand_id, company['name'], rand_dt, rand_usr])
    COMPANIES.append(objects.Customer(rand_id, rand_dt, company['credit_worthyness']))

kna1 = pd.DataFrame(kna1_temp_list, columns=Kna1_cols)

In [7]:
vbak_temp_list = []
vbap_temp_list = []
likp_temp_list = []
lips_temp_list = []
cdhdr_changes_temp_list = []
cdpos_changes_temp_list = []
vbuk_temp_list = []
mkpf_temp_list = []
mseg_temp_list = []
vbrk_temp_list = []
vbrp_temp_list = []

for i in range(10_000):
    vbak_obj = objects.SalesOrder(users=USERS, companies=COMPANIES, materials=MATERIALS)
    billing_dev_obj = objects.BillingDeviation(vbak=vbak_obj)
    cancel_return_dev_obj = objects.CancelOrReturn(vbak=vbak_obj)

    activities = [
        vbak_obj.generate_delivery_doc, 
        vbak_obj.release_delivery, 
        billing_dev_obj.set_billing_block, 
        vbak_obj.ship_goods, 
        billing_dev_obj.remove_billing_block, 
        vbak_obj.send_invoice, 
        cancel_return_dev_obj.cancel_order, 
        vbak_obj.receive_delivery_confirmation, 
        cancel_return_dev_obj.return_goods, 
        vbak_obj.clear_invoice,
        'end'
    ]

    step = 0
    while step < len(activities) - 1:
        # take a step
        activities[step]()
        vbak_obj.transition_matrix = vbak_obj.transition_matrix / vbak_obj.transition_matrix.sum(axis=1, keepdims=True)
                
        # step = next_step
        step = np.random.choice(len(activities), p=vbak_obj.transition_matrix[step]) # select another step

        # break if end
        if step == len(activities) - 1:
                break

    vbuk_temp_list.append(vbak_obj.vbuk)
    vbak_temp_list.append([values.mandt, vbak_obj.vbeln, vbak_obj.erdat, vbak_obj.ernam, vbak_obj.kunnr, vbak_obj.netwr, vbak_obj.vsbed, vbak_obj.faksk])
    vbap_temp_list += vbak_obj.vbap_temp_list
    likp_temp_list += vbak_obj.likp_temp_list
    lips_temp_list += vbak_obj.lips_temp_list
    cdhdr_changes_temp_list += vbak_obj.cdhdr_changes_temp_list
    cdpos_changes_temp_list += vbak_obj.cdpos_changes_temp_list
    mkpf_temp_list += vbak_obj.mkpf_temp_list
    mseg_temp_list += vbak_obj.mseg_temp_list
    vbrk_temp_list += vbak_obj.vbrk_temp_list
    vbrp_temp_list += vbak_obj.vbrp_temp_list

In [8]:
vbak = pd.DataFrame(vbak_temp_list, columns=vbak_cols)
vbap = pd.DataFrame(vbap_temp_list, columns=vbap_cols)
likp = pd.DataFrame(likp_temp_list, columns=likp_cols)
lips = pd.DataFrame(lips_temp_list, columns=lips_cols)
cdhdr = pd.DataFrame(cdhdr_changes_temp_list, columns=cdhdr_cols)
cdpos = pd.DataFrame(cdpos_changes_temp_list, columns=cdpos_cols)
vbuk = pd.DataFrame(vbuk_temp_list, columns=vbuk_cols)
mkpf = pd.DataFrame(mkpf_temp_list, columns=mkpf_cols)
mseg = pd.DataFrame(mseg_temp_list, columns=mseg_cols)
vbrk = pd.DataFrame(vbrk_temp_list, columns=vbrk_cols)
vbrp = pd.DataFrame(vbrp_temp_list, columns=vbrp_cols)

##### Save csv

In [9]:
version = str(datetime.now())
os.mkdir(f'data/{version}')
cdhdr.to_csv(f'data/{version}/cdhdr.csv', index=False) 
cdpos.to_csv(f'data/{version}/cdpos.csv', index=False) 
mara.to_csv(f'data/{version}/mara.csv', index=False) 
usr01.to_csv(f'data/{version}/usr01.csv', index=False) 
kna1.to_csv(f'data/{version}/kna1.csv', index=False) 
vbak.to_csv(f'data/{version}/vbak.csv', index=False) 
vbap.to_csv(f'data/{version}/vbap.csv', index=False) 
likp.to_csv(f'data/{version}/likp.csv', index=False) 
lips.to_csv(f'data/{version}/lips.csv', index=False) 
vbuk.to_csv(f'data/{version}/vbuk.csv', index=False) 
mkpf.to_csv(f'data/{version}/mkpf.csv', index=False) 
mseg.to_csv(f'data/{version}/mseg.csv', index=False) 
vbrk .to_csv(f'data/{version}/vbrk.csv', index=False) 
vbrp .to_csv(f'data/{version}/vbrp.csv', index=False) 

In [10]:
vbap

,MANDT,VBELN,POSNR,KWMENG,MATNR,NETWR
0,SC1,6717907c-0a96-4dc1-8da9-3fe84fa32dc4,0,132,1d001dbd-b1ae-412c-ac37-8dde61062942,924.0
1,SC1,6717907c-0a96-4dc1-8da9-3fe84fa32dc4,1,111,ffbb5be3-ceb0-42fb-a3d9-c385233241a7,499.5
2,SC1,6717907c-0a96-4dc1-8da9-3fe84fa32dc4,2,39,ce60230b-f9fd-4675-9415-7ff6a1ece0f6,78.0
3,SC1,6717907c-0a96-4dc1-8da9-3fe84fa32dc4,3,85,24dcf86f-5a51-4c94-b1a7-6148d5ffc949,425.0
4,SC1,6717907c-0a96-4dc1-8da9-3fe84fa32dc4,4,27,ea0adffe-62f5-46b3-bc20-5baa5b86178c,210.6
...,...,...,...,...,...,...
99043,SC1,a8ed9fff-2e0f-44f3-ae4a-5d6572fc6e84,4,47,ea0adffe-62f5-46b3-bc20-5baa5b86178c,366.6
99044,SC1,a8ed9fff-2e0f-44f3-ae4a-5d6572fc6e84,5,78,63711963-e4f7-4b0b-8105-5a125ecb55f0,624.0
99045,SC1,a8ed9fff-2e0f-44f3-ae4a-5d6572fc6e84,6,108,0e3f00ea-821f-4e0e-be5c-db1fb7414f4e,270.0
99046,SC1,a8ed9fff-2e0f-44f3-ae4a-5d6572fc6e84,7,29,dd807cff-d72d-44d4-8ca1-c45bbf4c1823,197.2
